# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the same account you used in the lab (hvt201)  and query the database that you were querying in class (citibike_feb_2015)

In [2]:
SQL_SOURCE = 'https://kwp225.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-2-a06cf3a40a9b>, line 23)

### Task 1 — Familiarize with SQL Clauses

• Sort data by start_station_id, tripduration
- Only checking trips with duration <= 3 hours

• Only show the top/last 10 records (aka head and tail in SQL)

• List all unique start_station_id values

• Aggregation functions:
- Count the number of trips (aka wc -l in SQL)
- Find the average/min/max trip duration

In [3]:
# Sort data by start_station_id,tripduration , only checking trips with duration <= 3 hours
task1_1 = '''
SELECT *
FROM citibike
WHERE tripduration <= 10800
ORDER BY start_station_id, tripduration
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task1_1)), sep=',')

NameError: name 'pd' is not defined

In [ ]:
# Only show the top 10 records
task1_2a = '''
SELECT *
FROM citibike
WHERE tripduration <= 10800
ORDER BY tripduration 
LIMIT 10
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task1_2a)), sep=',')

In [ ]:
# Only show the last 10 records
task1_2b = '''
SELECT *
FROM citibike
WHERE tripduration <= 10800
ORDER BY tripduration DESC
LIMIT 10
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task1_2b)), sep=',')

In [ ]:
# List all unique start_station_id values
task1_3 = '''
SELECT DISTINCT start_station_id
FROM citibike
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task1_3)), sep=',')

In [ ]:
# Count the number of trips 
task1_4a = '''
SELECT COUNT(*)
FROM citibike
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task1_4a)), sep=',')

In [ ]:
# Find the average/min/max trip duration 
task1_4b = '''
SELECT avg(tripduration),
       min(tripduration),
       max(tripduration)
FROM citibike
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task1_4b)), sep=',')

### Task 2 — Working with date/time

• Selecting trips started on Feb-02-2015 only

• Selecting trips started on the weekends
- What are average trip duration during weekends?

• Can we do the same for weekday?


In [ ]:
# Selecting trips started on Feb-02-2015 only
task2_1 = '''
SELECT *
FROM citibike
WHERE starttime >= '2015-02-02 00:00'
      AND starttime < '2015-02-03 00:00'
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task2_1)), sep=',')

In [ ]:
# Selecting trips started on the weekends
# What are average trip duration during weekends?
task2_2a = '''
SELECT *
FROM citibike
WHERE extract(DOW FROM starttime) IN (0,6)
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task2_2a)), sep=',')

In [ ]:
# What are average trip duration during weekends?
task2_2b = '''
SELECT avg(tripduration)
FROM citibike
WHERE extract(DOW FROM starttime) IN (0,6)
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task2_2b)), sep=',')

In [ ]:
# Can we do the same for weekday?
# Selecting trips started on the weekdays
task2_3a = '''
SELECT *
FROM citibike
WHERE extract(DOW FROM starttime) NOT IN (0,6)
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task2_3a)), sep=',')

In [ ]:
# Can we do the same for weekday?
# What are average trip duration during weekdays?
task2_3b = '''
SELECT avg(tripduration)
FROM citibike
WHERE extract(DOW FROM starttime) NOT IN (0,6)
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task2_3b)), sep=',')

### Task 3 — Working with Space
• Showing the list of start station locations
- Using GROUP BY

• Showing the number of trips started per station

• … but only for stations within 500m of Time Square!
- The coordinates of Time Square is (40.7577,-73.9857)

In [ ]:
# Showing the list of start station locations using GROUP BY
task3_1 = '''
SELECT start_station_latitude, start_station_longitude
FROM citibike
GROUP BY start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task3_1)), sep=',')

In [ ]:
# Showing the number of trips started per station
task3_2 = '''
SELECT start_station_id, COUNT(*) as numTrips
FROM citibike
GROUP BY start_station_id, start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task3_2)), sep=',')

In [ ]:
# Showing the number of trips started per station but only for stations within 500m of Time Square!
# The coordinates of Time Square is (40.7577,-73.9857)
task3_3 = '''
SELECT start_station_id, 
       start_station_name, 
       COUNT(*) as numTrips

FROM citibike

WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500)
                            
GROUP BY start_station_id, start_station_name, start_station_latitude, start_station_longitude
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task3_3)), sep=',')

### Task 4 — Putting it all together
• Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare!

• Extra: create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines. 

In [ ]:
# Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare!
task4_1 = '''
SELECT start_station_id, 
       start_station_name, 
       avg(tripduration) as avgTripTime

FROM citibike

WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500) 
      AND extract(DOW FROM starttime) IN (0,6)
                 
GROUP BY start_station_id, start_station_name, start_station_latitude, start_station_longitude

ORDER BY avgTripTime DESC

LIMIT 1

'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task4_1)), sep=',')

In [ ]:
# Extra: create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours. 
# The number of trips per each pair of stations are output as attributes of these lines.
task4_2 = '''
SELECT ST_Makeline(
        CDB_TransformToWebmercator(
            CDB_LatLng(start_station_latitude, start_station_longitude)),
        CDB_TransformToWebmercator(
            CDB_LatLng(end_station_latitude, end_station_longitude))
        ) as the_geom_webmercator,
        MIN(cartodb_id) as cartodb_id
        
FROM citibike

WHERE ST_DWithin(CDB_LatLng(start_station_latitude,
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500)
      AND tripduration <=7200
                            
GROUP BY start_station_id, start_station_latitude, start_station_longitude,
         end_station_id, end_station_latitude, end_station_longitude
         
ORDER BY start_station_id
'''

pd.read_csv(StringIO.StringIO(queryCartoDB(task4_2)), sep=',')

### Link to Carto Map of Task 4.2



https://kwp225.carto.com/builder/ebf39417-0af9-44a6-b052-fe4949e56e3e/embed

## ASH -100. Extra Ctedit 100